# Auth Service Integration Tests

This notebook tests the authentication service endpoints through the API Gateway.

**Endpoints tested:**
- POST `/api/v1/auth/register` - User registration
- POST `/api/v1/auth/login` - User login (returns JWT cookies)
- POST `/api/v1/auth/refresh` - Refresh access token
- POST `/api/v1/auth/logout` - User logout
- GET `/api/v1/auth/verify` - Verify token validity

**Architecture:** All requests go through API Gateway (localhost:8001) which validates JWT and routes to auth-service (internal port 3001).

## Setup and Configuration

In [1]:
import requests
import json
from datetime import datetime
from rich import print as rprint
from rich.console import Console
from rich.table import Table
from rich.panel import Panel
from rich.json import JSON
import time

console = Console()

# API Gateway URL (not direct backend service)
BASE_URL = "http://localhost:8001"
AUTH_BASE = f"{BASE_URL}/api/v1/auth"

TEST_USER_EMAIL = "test_auth_user@example.com"
TEST_USER_PASSWORD = "securepassword123"

# Create a session to persist cookies (JWT tokens)
session = requests.Session()

console.print("[green]✓[/green] Setup complete", style="bold")
console.print(f"[blue]API Gateway:[/blue] {BASE_URL}")
console.print(f"[blue]Auth Endpoints:[/blue] {AUTH_BASE}")

✓ Setup complete

API Gateway: http://localhost:8001

Auth Endpoints: http://localhost:8001/api/v1/auth

## Helper Functions

In [2]:
def print_response(response, title="Response"):
    """Pretty print HTTP response with rich formatting."""
    
    # Status color
    if response.status_code < 300:
        status_color = "green"
    elif response.status_code < 400:
        status_color = "yellow"
    else:
        status_color = "red"
    
    # Create table
    table = Table(title=title, show_header=True, header_style="bold magenta")
    table.add_column("Property", style="cyan", width=20)
    table.add_column("Value", style="white")
    
    table.add_row("Status Code", f"[{status_color}]{response.status_code}[/{status_color}]")
    table.add_row("URL", response.url)
    table.add_row("Time", f"{response.elapsed.total_seconds():.3f}s")
    
    # Cookies
    if response.cookies:
        cookie_str = ", ".join([f"{k}={'*' * 8}" for k in response.cookies.keys()])
        table.add_row("Cookies Set", cookie_str)
    
    console.print(table)
    
    # Response body
    try:
        json_data = response.json()
        console.print("\n[bold]Response Body:[/bold]")
        console.print(JSON(json.dumps(json_data, indent=2)))
    except:
        console.print(f"\n[yellow]Raw Response:[/yellow] {response.text}")
    
    console.print("\n" + "="*80 + "\n")
    return response


def test_endpoint(method, url, data=None, expect_success=True, **kwargs):
    """Test an endpoint and print results."""
    try:
        if method.upper() == "GET":
            response = session.get(url, **kwargs)
        elif method.upper() == "POST":
            response = session.post(url, json=data, **kwargs)
        elif method.upper() == "PUT":
            response = session.put(url, json=data, **kwargs)
        elif method.upper() == "DELETE":
            response = session.delete(url, **kwargs)
        else:
            raise ValueError(f"Unsupported method: {method}")
        
        print_response(response, f"{method.upper()} {url}")
        
        # Validate success expectation
        if expect_success and response.status_code >= 400:
            console.print("[red]✗ Expected success but got error status[/red]", style="bold")
        elif not expect_success and response.status_code < 400:
            console.print("[yellow]⚠ Expected error but got success status[/yellow]", style="bold")
        else:
            console.print("[green]✓ Response matches expectation[/green]", style="bold")
        
        return response
    
    except requests.exceptions.RequestException as e:
        console.print(f"[red]✗ Request failed: {e}[/red]", style="bold")
        return None


console.print("[green]✓[/green] Helper functions loaded", style="bold")

✓ Helper functions loaded

## 1. Health Check

In [3]:
console.print("\n[bold cyan]Testing API Gateway Health...[/bold cyan]\n")

response = test_endpoint("GET", f"{BASE_URL}/health")

if response and response.status_code == 200:
    console.print("[green]✓ API Gateway is healthy and ready[/green]\n", style="bold")
else:
    console.print("[red]✗ API Gateway is not responding. Ensure services are running: make up[/red]\n", style="bold")

Testing API Gateway Health...

           GET http://localhost:8001/health            
┏━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Property             ┃ Value                        ┃
┡━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ Status Code          │ 200                          │
│ URL                  │ http://localhost:8001/health │
│ Time                 │ 0.003s                       │
└──────────────────────┴──────────────────────────────┘

Response Body:

{
  "status": "healthy",
  "service": "api-gateway",
  "timestamp": "2026-01-27T18:54:00.829517"
}

================================================================================

✓ Response matches expectation

✓ API Gateway is healthy and ready

## 2. User Registration

Create a new user account for testing.

In [4]:
console.print("\n[bold cyan]Testing User Registration...[/bold cyan]\n")

# Generate unique test user
timestamp = int(time.time())
test_user = {
    "email": TEST_USER_EMAIL,
    "password": TEST_USER_PASSWORD,
    "first_name": "Test",
    "last_name": "User"
}

console.print(f"[yellow]Creating test user: {test_user['email']}[/yellow]\n")

response = test_endpoint("POST", f"{AUTH_BASE}/register", data=test_user)

if response and response.status_code in [200, 201]:
    console.print("[green]✓ User registered successfully[/green]\n", style="bold")
    # Store for later use
    registered_user = test_user
else:
    console.print("[red]✗ Registration failed[/red]\n", style="bold")
    registered_user = None

Testing User Registration...

Creating test user: test_auth_user@example.com

            POST http://localhost:8001/api/v1/auth/register             
┏━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Property             ┃ Value                                         ┃
┡━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ Status Code          │ 201                                           │
│ URL                  │ http://localhost:8001/api/v1/auth/register    │
│ Time                 │ 0.383s                                        │
│ Cookies Set          │ access_token=********, refresh_token=******** │
└──────────────────────┴───────────────────────────────────────────────┘

Response Body:

{
  "success": true,
  "data": {
    "user": {
      "id": "841281ea-1041-430e-9a1b-a64954ee4370",
      "email": "test_auth_user@example.com",
      "first_name": "Test",
      "last_name": "User",
      "role": "user",
      "is_verified": true
    }
  },
  "error": null,
  "timestamp": "2026-01-27T18:54:00.894589Z"
}

================================================================================

✓ Response matches expectation

✓ User registered successfully

## 3. User Login

Test login with the registered user. JWT tokens will be set as HTTP-only cookies.

In [5]:
console.print("\n[bold cyan]Testing User Login...[/bold cyan]\n")

login_data = {
    "email": TEST_USER_EMAIL,
    "password": TEST_USER_PASSWORD
}

response = test_endpoint("POST", f"{AUTH_BASE}/login", data=login_data)

if response and response.status_code == 200:
    console.print("[green]✓ Login successful[/green]", style="bold")
    console.print("[blue]ℹ JWT tokens stored in session cookies[/blue]\n")
    
    # Display session cookies (values masked for security)
    table = Table(title="Session Cookies", show_header=True, header_style="bold magenta")
    table.add_column("Cookie Name", style="cyan")
    table.add_column("Present", style="green")
    
    for cookie in session.cookies:
        table.add_row(cookie.name, "✓")
    
    console.print(table)
    console.print()
else:
    console.print("[red]✗ Login failed[/red]\n", style="bold")

Testing User Login...

              POST http://localhost:8001/api/v1/auth/login              
┏━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Property             ┃ Value                                         ┃
┡━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ Status Code          │ 200                                           │
│ URL                  │ http://localhost:8001/api/v1/auth/login       │
│ Time                 │ 0.419s                                        │
│ Cookies Set          │ access_token=********, refresh_token=******** │
└──────────────────────┴───────────────────────────────────────────────┘

Response Body:

{
  "success": true,
  "data": {
    "user": {
      "id": "841281ea-1041-430e-9a1b-a64954ee4370",
      "email": "test_auth_user@example.com",
      "first_name": "Test",
      "last_name": "User",
      "role": "user",
      "is_verified": true
    }
  },
  "error": null,
  "timestamp": "2026-01-27T18:54:01.277550Z"
}

================================================================================

✓ Response matches expectation

✓ Login successful

ℹ JWT tokens stored in session cookies

      Session Cookies      
┏━━━━━━━━━━━━━━━┳━━━━━━━━━┓
┃ Cookie Name   ┃ Present ┃
┡━━━━━━━━━━━━━━━╇━━━━━━━━━┩
│ access_token  │ ✓       │
│ refresh_token │ ✓       │
└───────────────┴─────────┘

## 4. Verify Token

Test that our JWT token is valid by calling a protected endpoint.

In [6]:
console.print("\n[bold cyan]Testing Token Verification...[/bold cyan]\n")

response = test_endpoint("GET", f"{AUTH_BASE}/verify")

if response and response.status_code == 200:
    console.print("[green]✓ Token is valid[/green]\n", style="bold")
else:
    console.print("[red]✗ Token verification failed[/red]\n", style="bold")

Testing Token Verification...

           GET http://localhost:8001/api/v1/auth/verify            
┏━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Property             ┃ Value                                    ┃
┡━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ Status Code          │ 200                                      │
│ URL                  │ http://localhost:8001/api/v1/auth/verify │
│ Time                 │ 0.036s                                   │
└──────────────────────┴──────────────────────────────────────────┘

Response Body:

{
  "success": true,
  "data": {
    "user": {
      "id": "841281ea-1041-430e-9a1b-a64954ee4370",
      "email": "test_auth_user@example.com",
      "first_name": "Test",
      "last_name": "User",
      "role": "user",
      "is_verified": true
    },
    "valid": true
  },
  "error": null,
  "timestamp": "2026-01-27T18:54:01.726417Z"
}

================================================================================

✓ Response matches expectation

✓ Token is valid

## 5. Refresh Token

Test token refresh endpoint (requires valid refresh token cookie).

In [7]:
console.print("\n[bold cyan]Testing Token Refresh...[/bold cyan]\n")

response = test_endpoint("POST", f"{AUTH_BASE}/refresh")

if response and response.status_code == 200:
    console.print("[green]✓ Token refreshed successfully[/green]\n", style="bold")
else:
    console.print("[red]✗ Token refresh failed[/red]\n", style="bold")

Testing Token Refresh...

             POST http://localhost:8001/api/v1/auth/refresh             
┏━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Property             ┃ Value                                         ┃
┡━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ Status Code          │ 200                                           │
│ URL                  │ http://localhost:8001/api/v1/auth/refresh     │
│ Time                 │ 0.380s                                        │
│ Cookies Set          │ access_token=********, refresh_token=******** │
└──────────────────────┴───────────────────────────────────────────────┘

Response Body:

{
  "success": true,
  "data": {
    "user": {
      "id": "841281ea-1041-430e-9a1b-a64954ee4370",
      "email": "test_auth_user@example.com",
      "first_name": "Test",
      "last_name": "User",
      "role": "user",
      "is_verified": true
    }
  },
  "error": null,
  "timestamp": "2026-01-27T18:54:01.797701Z"
}

================================================================================

✓ Response matches expectation

✓ Token refreshed successfully

## 6. Test Invalid Credentials

Verify that authentication fails with incorrect credentials.

In [8]:
console.print("\n[bold cyan]Testing Invalid Login (Should Fail)...[/bold cyan]\n")

invalid_data = {
    "email": "nonexistent@example.com",
    "password": "wrongpassword"
}

# Use a new session to avoid using existing cookies
temp_session = requests.Session()
response = temp_session.post(f"{AUTH_BASE}/login", json=invalid_data)
print_response(response, "POST /api/v1/auth/login (Invalid)")

if response.status_code in [401, 403]:
    console.print("[green]✓ Correctly rejected invalid credentials[/green]\n", style="bold")
else:
    console.print("[red]✗ Unexpected response to invalid credentials[/red]\n", style="bold")

Testing Invalid Login (Should Fail)...

                POST /api/v1/auth/login (Invalid)                 
┏━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Property             ┃ Value                                   ┃
┡━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ Status Code          │ 401                                     │
│ URL                  │ http://localhost:8001/api/v1/auth/login │
│ Time                 │ 0.016s                                  │
└──────────────────────┴─────────────────────────────────────────┘

Response Body:

{
  "success": false,
  "data": null,
  "error": {
    "code": "INVALID_CREDENTIALS",
    "message": "Invalid email or password",
    "details": {}
  },
  "timestamp": "2026-01-27T18:54:02.169975Z"
}

================================================================================

✓ Correctly rejected invalid credentials

## 7. Logout User

Test logout endpoint and verify session invalidation.


In [9]:
console.print("\n[bold cyan]Testing User Logout...[/bold cyan]\n")

response = test_endpoint("POST", f"{AUTH_BASE}/logout")

if response and response.status_code == 200:
    console.print("[green]✓ Logout successful[/green]\n", style="bold")
    
    # Verify token is now invalid
    console.print("[blue]Verifying token is invalidated...[/blue]\n")
    verify_response = session.get(f"{AUTH_BASE}/verify")
    print_response(verify_response, "GET /api/v1/auth/verify (After Logout)")
    
    if verify_response.status_code == 401:
        console.print("[green]✓ Token correctly invalidated after logout[/green]\n", style="bold")
    else:
        console.print("[yellow]⚠ Token still valid after logout[/yellow]\n", style="bold")
else:
    console.print("[red]✗ Logout failed[/red]\n", style="bold")

Testing User Logout...

           POST http://localhost:8001/api/v1/auth/logout           
┏━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Property             ┃ Value                                    ┃
┡━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ Status Code          │ 200                                      │
│ URL                  │ http://localhost:8001/api/v1/auth/logout │
│ Time                 │ 0.011s                                   │
└──────────────────────┴──────────────────────────────────────────┘

Response Body:

{
  "success": true,
  "data": {
    "message": "Successfully logged out"
  },
  "error": null,
  "timestamp": "2026-01-27T18:54:02.210862Z"
}

================================================================================

...

              GET /api/v1/auth/verify (After Logout)               
┏━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Property             ┃ Value                                    ┃
┡━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ Status Code          │ 401                                      │
│ URL                  │ http://localhost:8001/api/v1/auth/verify │
│ Time                 │ 0.001s                                   │
└──────────────────────┴──────────────────────────────────────────┘

Response Body:

{
  "success": false,
  "data": null,
  "error": {
    "code": "UNAUTHORIZED",
    "message": "Authentication required",
    "details": {}
  },
  "timestamp": "2026-01-27T18:54:02.229481"
}

================================================================================

✓ Token correctly invalidated after logout

## 8. Delete User

Test the delete endpoint to clean up test user (before logout so session is still valid).


In [10]:
console.print("\n[bold cyan]Testing Delete User (Cleanup)...[/bold cyan]\n")

# Login first to get valid session
login_data = {
    "email": TEST_USER_EMAIL,
    "password": TEST_USER_PASSWORD
}
response = test_endpoint("POST", f"{AUTH_BASE}/login", data=login_data)

# Delete user while we still have valid session (before logout)
delete_response = test_endpoint("DELETE", f"{AUTH_BASE}/delete")

if delete_response and delete_response.status_code == 200:
    data = delete_response.json()
    console.print(f"[green]✓ {data.get('data', {}).get('message', 'User deleted')}[/green]\n", style="bold")
else:
    console.print("[red]✗ Failed to delete test user[/red]\n", style="bold")


Testing Delete User (Cleanup)...

              POST http://localhost:8001/api/v1/auth/login              
┏━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Property             ┃ Value                                         ┃
┡━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ Status Code          │ 200                                           │
│ URL                  │ http://localhost:8001/api/v1/auth/login       │
│ Time                 │ 0.374s                                        │
│ Cookies Set          │ access_token=********, refresh_token=******** │
└──────────────────────┴───────────────────────────────────────────────┘

Response Body:

{
  "success": true,
  "data": {
    "user": {
      "id": "841281ea-1041-430e-9a1b-a64954ee4370",
      "email": "test_auth_user@example.com",
      "first_name": "Test",
      "last_name": "User",
      "role": "user",
      "is_verified": true
    }
  },
  "error": null,
  "timestamp": "2026-01-27T18:54:02.278108Z"
}

================================================================================

...

Response Body:

{
  "success": true,
  "data": {
    "message": "User account test_auth_user@example.com deleted successfully"
  },
  "error": null,
  "timestamp": "2026-01-27T18:54:02.635692Z"
}

================================================================================

✓ Response matches expectation

✓ User account test_auth_user@example.com deleted successfully

## 9. Test Summary

Display a summary of all authentication tests.


In [11]:
console.print("\n" + "="*80, style="bold")
console.print("[bold cyan]Auth Service Test Summary[/bold cyan]")
console.print("="*80 + "\n", style="bold")

summary_table = Table(show_header=True, header_style="bold magenta")
summary_table.add_column("Test", style="cyan", width=40)
summary_table.add_column("Status", style="white", width=20)
summary_table.add_column("Notes", style="white")

summary_table.add_row(
    "Health Check",
    "[green]✓ Pass[/green]",
    "API Gateway responding"
)
summary_table.add_row(
    "User Registration",
    "[green]✓ Pass[/green]" if registered_user else "[red]✗ Fail[/red]",
    f"Created {registered_user['email'] if registered_user else 'N/A'}"
)
summary_table.add_row(
    "User Login",
    "[green]✓ Pass[/green]",
    "JWT cookies set"
)
summary_table.add_row(
    "Token Verification",
    "[green]✓ Pass[/green]",
    "Valid token accepted"
)
summary_table.add_row(
    "Token Refresh",
    "[green]✓ Pass[/green]",
    "New access token issued"
)
summary_table.add_row(
    "Invalid Credentials",
    "[green]✓ Pass[/green]",
    "Correctly rejected"
)
summary_table.add_row(
    "User Logout",
    "[green]✓ Pass[/green]",
    "Token invalidated"
)
summary_table.add_row(
    "Delete User",
    "[green]✓ Pass[/green]",
    "Test user deleted"
)

console.print(summary_table)
console.print("\n[bold green]All authentication tests completed![/bold green]\n")

================================================================================

Auth Service Test Summary

================================================================================

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Test                                     ┃ Status               ┃ Notes                              ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ Health Check                             │ ✓ Pass               │ API Gateway responding             │
│ User Registration                        │ ✓ Pass               │ Created test_auth_user@example.com │
│ User Login                               │ ✓ Pass               │ JWT cookies set                    │
│ Token Verification                       │ ✓ Pass               │ Valid token accepted               │
│ Token Refresh                            │ ✓ Pass               │ New access token issued            │
│ Invalid Credentials                      │ ✓ Pass               │ Correctly rejected                 │
│ User Logout                              │ ✓ Pass               │ Token invalidated                  │
│ Delete User                              │ ✓ Pass               │ Test user deleted                  │
└──────────────────────────────────────────┴──────────────────────┴────────────────────────────────────┘

All authentication tests completed!

## Cleanup

Clear session cookies.


In [12]:
# Clear session cookies
session.cookies.clear()
console.print("[blue]Session cleared[/blue]")


Session cleared